# I. Setup

In [ ]:
%pip install ortools
%pip install pandas
%pip install numpy

In [ ]:
from ortools.sat.python import cp_model
import pandas as pd
import numpy as np

### Model selection: 'strict' or 'regulated'
### Dataset: 'data.csv' or 'non-iid-data.csv'


In [3]:
model = 'strict'
data_file = 'data.csv'

# II. Data Preprocessing

Let $T_1, T_2, T_3$ respectively be sets of employees of types **Tier I Doctor**, **Tier II Doctor**, and **Nurses**; $D_1, D_2, D_3, D_4$ respectively be sets of employees in departments **Internal**, **External**, **Emergency** and **Clinical**; and $S$ be set of employees which are **Seniors**.

In [4]:
# Read the data from the CSV file
data = pd.read_csv(data_file)

# Map for departments and types
department_map = {"Internal": 1, "External": 2, "Emergency": 3, "Clinical": 4}
type_map = {"Tier I Doctor": 1, "Tier II Doctor": 2, "Nurse": 3}

# Create hospital list from data
hospital = []
for index, row in data.iterrows():
    hospital.append((row['Tên'], row['Điểm thâm niên'], row['Khoa'], row['Loại']))

# Calculate the threshold to be considered `senior
senior_threshold = int(np.ceil(np.percentile(data['Điểm thâm niên'], 75)))

# Initialize lists for different groups
T1 = []
T2 = []
T3 = []
D1 = []
D2 = []
D3 = []
D4 = []
S = []

# Populate the lists based on conditions
for i, h in enumerate(hospital):
    if h[3] == 1:
        T1.append(i)
    elif h[3] == 2:
        T2.append(i)
    elif h[3] == 3:
        T3.append(i)
    
    if h[2] == 1:
        D1.append(i)
    elif h[2] == 2:
        D2.append(i)
    elif h[2] == 3:
        D3.append(i)
    elif h[2] == 4:
        D4.append(i)
    
    if h[1] >= senior_threshold:
        S.append(i)


# III. Problem Modeling

## A. Model

Let $X_{i, j}$ be a decision variable which is set to $1$ if employee at index $i$ works on shift $j$.

In [5]:
# Number of hospitals and shifts
num_hospital = len(hospital)
num_shifts = 61

# Initialize the solver
solver = cp_model.CpModel()

X = {}
for i in range(num_hospital):
    for j in range(num_shifts):
        X[i, j] = solver.NewIntVar(0, 1, f'X_{i}_{j}')

### i. Hard constraint

The hard constraint can be describe as
$$\sum_{i \in T_1} X_{i, j} = 1, \ \sum_{i \in T_2} X_{i, j} = 1, \ \sum_{i \in T_3} X_{i, j} = 2,\ \forall j$$

In [6]:
for j in range(num_shifts):
    solver.Add(sum((X[i, j]) for i in T1) == 1)
    solver.Add(sum((X[i, j]) for i in T2) == 1)
    solver.Add(sum((X[i, j]) for i in T3) == 2)

### ii. Soft constraints

For the first soft constraint, which is each shift should have a variety of departments, we have
$$\sum_{i \in D_1} X_{i, j} = 1, \ \sum_{i \in D_2} X_{i, j} = 1, \ \sum_{i \in D_3} X_{i, j} = 1, \ \sum_{i \in D_4} X_{i, j} = 1,\ \forall j$$

In [7]:
if model == 'strict':
    for j in range(num_shifts):
        solver.Add(sum(X[i, j] for i in D1) == 1)
        solver.Add(sum(X[i, j] for i in D2) == 1)
        solver.Add(sum(X[i, j] for i in D3) == 1)
        solver.Add(sum(X[i, j] for i in D4) == 1)

If in case we want to harmonize the number of total departments and the fair distribution of shifts per person, we can adjust our above constraint like as below. We have chosen departments $3$ and $4$ since it has the most employess.

In [8]:
if model == 'regulated':
    for j in range(num_shifts):
        solver.Add(sum(X[i, j] for i in D4) == 1)
        solver.Add(sum(X[i, j] for i in D3) == 1)

For seniority constraint, we have
$$\sum_{i \in S} X_{i, j} \ge 1, \ \forall j$$
Or for strictlier constraint, we can say
$$\sum_{i \in S} X_{i, j} = 1, \ \forall j$$

In [9]:
for j in range(num_shifts):
    solver.Add(sum(X[i, j] for i in S) == 1)

For evenly spread in shifts, notice that the department constraints inspires us to evenly spread employees by average shifts per department, we have
$$\text{lower}_i \le \sum_{j \in \text{shifts}}X[i, j] \le \text{upper}_i$$
where $i$ ranges in sets of $D_1, D_2, D_3$ and $D_4$.

In [10]:
# Calculate average shifts for each department
average_shift_dept1 = round(num_shifts / len(D1))
average_shift_dept2 = round(num_shifts / len(D2))
average_shift_dept3 = round(num_shifts / len(D3))
average_shift_dept4 = round(num_shifts / len(D4))

# Add constraints to ensure fair shift distribution across departments
for i in D1:
    solver.Add(sum(X[i, j] for j in range(num_shifts)) >= average_shift_dept1)
    solver.Add(sum(X[i, j] for j in range(num_shifts)) <= average_shift_dept1 + 1)

for i in D2:
    solver.Add(sum(X[i, j] for j in range(num_shifts)) >= average_shift_dept2 - 1)
    solver.Add(sum(X[i, j] for j in range(num_shifts)) <= average_shift_dept2)

for i in D3:
    solver.Add(sum(X[i, j] for j in range(num_shifts)) >= average_shift_dept3 - 1)
    solver.Add(sum(X[i, j] for j in range(num_shifts)) <= average_shift_dept3)

for i in D4:
    solver.Add(sum(X[i, j] for j in range(num_shifts)) >= average_shift_dept4 - 1)
    solver.Add(sum(X[i, j] for j in range(num_shifts)) <= average_shift_dept4 + 1)

### iii. Side constraints

This side constraint states that in a radius of $7$ days, an employee can only work for less than or equal to $1$ shift.
$$\sum_{k=j}^{j+7}X_{i, k} \le 1, \ j \in [0, \text{num\_shifts} - 7], i \in H$$

In [11]:
for i in range(num_hospital):
    for j in range(0, num_shifts - 7):
        solver.Add(sum(X[i, k] for k in range(j, j+7)) <= 1)

This constraint aims to evenly spread the total for to each shift, including this constraint slows down the process.
$$\text{score}_l \le \sum_{i\in H}X_{i, j} \le \text{score}_u, \ \forall i \in \text{shifts}$$

In [12]:
for j in range(num_shifts):
    solver.Add(sum(X[i, j] * hospital[i][1] for i in range(num_hospital)) >= 40)
    solver.Add(sum(X[i, j] * hospital[i][1] for i in range(num_hospital)) <= 46)

## B. Loss Function

In [13]:
# Calculate losses for each department
loss1 = sum(sum(X[i, j] for j in range(num_shifts)) for i in D1)
loss2 = sum(sum(X[i, j] for j in range(num_shifts)) for i in D2)
loss3 = sum(sum(X[i, j] for j in range(num_shifts)) for i in D3)
loss4 = sum(sum(X[i, j] for j in range(num_shifts)) for i in D4)

# Calculate losses for senior and non-senior staff in department Clinical
lossA = sum(sum(X[i, j] for j in range(num_shifts)) for i in D4 if i in S)
lossB = sum(sum(X[i, j] for j in range(num_shifts)) for i in D4 if i not in S)

This problem can be solve without a loss function, since it is a scheduling problem. But, as we see in all constraints, we aim to optimize the constraint about evenly spreaded days. This inspires us to use this as the loss function. The objective is to maximize the following value
$$\textbf{Sat} = \sum_{x\in{1, 2, 3, 4}}w_x\left(\sum_{i \in D_x}\sum_{j \in \text{shifts}}X_{i, j}\right)$$
Where $w_x$ are coefficients which we decide via practice.

In [14]:
if model == 'strict':
    solver.Minimize(lossB - lossA)

And in case we opt to harmonize total of department and fair distribution, we can modify our loss function as

In [15]:
if model == 'regulated':
    solver.Maximize(9 * loss1 + 18 * loss2 + 16 * loss3 + 4 * loss4)

# IV. Evaluation

In [16]:
solution = cp_model.CpSolver()
status = solution.Solve(solver)

In [17]:
schedule = []
if status == cp_model.OPTIMAL:
    print('Objective value: ', solution.ObjectiveValue())
    print('Solution found:')

    for j in range(num_shifts):
        shift = {"Tier I": None, "Tier II": None, "Nurse 1": None, "Nurse 2": None}
        nurses_count = 0
        for i in range(num_hospital):
            if solution.Value(X[i, j]) == 1:
                staff_type = hospital[i][3]
                staff_name = hospital[i][0]
                
                # Assign staff to the appropriate role
                if staff_type == 1:
                    shift["Tier I"] = staff_name
                elif staff_type == 2:
                    shift["Tier II"] = staff_name
                else:
                    if nurses_count == 0:
                        shift["Nurse 1"] = staff_name
                        nurses_count += 1
                    else:
                        shift["Nurse 2"] = staff_name

        schedule.append(shift)

    # Display the schedule
    for idx, shift in enumerate(schedule):
        print(f"Shift {idx + 1}: {shift}")
else:
    print('No optimal solution found.')

Objective value:  13.0
Solution found:
Shift 1: {'Tier I': 'Hường', 'Tier II': 'Xuân', 'Nurse 1': 'Hoài', 'Nurse 2': 'Oanh'}
Shift 2: {'Tier I': 'Thư', 'Tier II': 'Hương', 'Nurse 1': 'Nam', 'Nurse 2': 'Ngọc '}
Shift 3: {'Tier I': 'Thuỷ', 'Tier II': 'Hà', 'Nurse 1': 'Tiến', 'Nurse 2': 'Văn Đức'}
Shift 4: {'Tier I': 'Tuấn', 'Tier II': 'Dũng', 'Nurse 1': 'Tuyến', 'Nurse 2': 'Hưng'}
Shift 5: {'Tier I': 'Xuân', 'Tier II': 'Dung', 'Nurse 1': 'Tơ', 'Nurse 2': 'Huệ'}
Shift 6: {'Tier I': 'Quân', 'Tier II': 'Gấm', 'Nurse 1': 'Dũng', 'Nurse 2': 'Thành'}
Shift 7: {'Tier I': 'Sơn', 'Tier II': 'Vân Anh', 'Nurse 1': 'Tú', 'Nurse 2': 'Hương'}
Shift 8: {'Tier I': 'Ngân Hà', 'Tier II': 'Hưng', 'Nurse 1': 'Phi', 'Nurse 2': 'Đông'}
Shift 9: {'Tier I': 'Linh', 'Tier II': 'Xuân', 'Nurse 1': 'Linh', 'Nurse 2': 'Lương'}
Shift 10: {'Tier I': 'Thư', 'Tier II': 'Linh', 'Nurse 1': 'Văn Đức', 'Nurse 2': 'Dung'}
Shift 11: {'Tier I': 'Hường', 'Tier II': 'Hương', 'Nurse 1': 'Thu', 'Nurse 2': 'Thảo'}
Shift 12: {'Tier 

We evaluate the result by the following objectives:
1. Number of hard constraints violated.
2. Average Department Diversity per Shift.
3. Seniority Inclusion Percentage.
4. Standard Deviation of Shifts per Person (Fair Distribution), which is evaluated via every person and in each departments.

In [18]:
def evaluate_schedule(schedule, hospital, seniority_threshold):
    # Initialize counters and lists
    hard_constraint_violations = 0
    department_diversity_counts = []
    seniority_inclusion_count = 0

    tier_i_doctors = [doc for doc in hospital if doc[3] == 1]
    tier_ii_doctors = [doc for doc in hospital if doc[3] == 2]
    nurses = [doc for doc in hospital if doc[3] == 3]

    # Evaluate each shift
    for shift in schedule:
        departments_present = set()
        senior_member_present = False

        # Check Tier I doctor
        if shift["Tier I"] is None:
            hard_constraint_violations += 1
        else:
            for doc in tier_i_doctors:
                if doc[0] == shift["Tier I"]:
                    departments_present.add(doc[2])
                    if doc[1] >= seniority_threshold:
                        senior_member_present = True

        # Check Tier II doctor
        if shift["Tier II"] is None:
            hard_constraint_violations += 1
        else:
            for doc in tier_ii_doctors:
                if doc[0] == shift["Tier II"]:
                    departments_present.add(doc[2])
                    if doc[1] >= seniority_threshold:
                        senior_member_present = True

        # Check nurses
        if shift["Nurse 1"] is None or shift["Nurse 2"] is None:
            hard_constraint_violations += 1
        else:
            for nurse in nurses:
                if nurse[0] == shift["Nurse 1"] or nurse[0] == shift["Nurse 2"]:
                    departments_present.add(nurse[2])
                    if nurse[1] >= seniority_threshold:
                        senior_member_present = True

        # Record department diversity count
        department_diversity_counts.append(len(departments_present))

        # Record seniority inclusion
        if senior_member_present:
            seniority_inclusion_count += 1

    # Evaluate standard deviation of shifts per person
    person_shifts = [sum(solution.Value(X[i, j]) for j in range(num_shifts)) for i in range(num_hospital)]
    dept_1_shifts = [sum(solution.Value(X[i, j]) for j in range(num_shifts)) for i in D1]
    dept_2_shifts = [sum(solution.Value(X[i, j]) for j in range(num_shifts)) for i in D2]
    dept_3_shifts = [sum(solution.Value(X[i, j]) for j in range(num_shifts)) for i in D3]
    dept_4_shifts = [sum(solution.Value(X[i, j]) for j in range(num_shifts)) for i in D4]

    average_shifts_per_person = sum(person_shifts) / len(person_shifts)
    average_shifts_dept_1 = sum(dept_1_shifts) / len(dept_1_shifts)
    average_shifts_dept_2 = sum(dept_2_shifts) / len(dept_2_shifts)
    average_shifts_dept_3 = sum(dept_3_shifts) / len(dept_3_shifts)
    average_shifts_dept_4 = sum(dept_4_shifts) / len(dept_4_shifts)

    # Calculate metrics
    average_department_diversity = sum(department_diversity_counts) / num_shifts
    seniority_inclusion_percentage = (seniority_inclusion_count / num_shifts) * 100
    fair_distribution_std_dev = (sum([(x - average_shifts_per_person) ** 2 for x in person_shifts]) / len(person_shifts)) ** 0.5
    fair_distribution_dept1 = (sum([(x - average_shifts_dept_1) ** 2 for x in dept_1_shifts]) / len(dept_1_shifts)) ** 0.5
    fair_distribution_dept2 = (sum([(x - average_shifts_dept_2) ** 2 for x in dept_2_shifts]) / len(dept_2_shifts)) ** 0.5
    fair_distribution_dept3 = (sum([(x - average_shifts_dept_3) ** 2 for x in dept_3_shifts]) / len(dept_3_shifts)) ** 0.5
    fair_distribution_dept4 = (sum([(x - average_shifts_dept_4) ** 2 for x in dept_4_shifts]) / len(dept_4_shifts)) ** 0.5

    # Print evaluation results
    print(f"Hard Constraint Violations: {hard_constraint_violations}")
    print(f"Average Department Diversity per Shift: {average_department_diversity}")
    print(f"Seniority Inclusion Percentage: {seniority_inclusion_percentage}%")
    print(f"Standard Deviation of Shifts per Person (Fair Distribution): {fair_distribution_std_dev}")
    print(f"By departments: Internal: {fair_distribution_dept1}, External: {fair_distribution_dept2}, Emergency: {fair_distribution_dept3}, Clinical: {fair_distribution_dept4}")

In [19]:
evaluate_schedule(schedule, hospital, senior_threshold)

Hard Constraint Violations: 0
Average Department Diversity per Shift: 4.0
Seniority Inclusion Percentage: 100.0%
Standard Deviation of Shifts per Person (Fair Distribution): 0.8645808232895293
By departments: Internal: 0.479157423749955, External: 0.29354352395090366, Emergency: 0.476280484787101, Clinical: 0.950206589107585
